# APS PMDI canister fluorescence

Post processing of X-ray canister fluorescence data with KI and BaSO4.
Third step - take collated scans and use model developed previously to perform signal trapping corrections.
Depends on geometry and position.

Data collected by Daniel Duke, Lingzhe Rao & Alan Kastengren
@ Advanced Photon Source, Argonne National Laboratory, Lemont, Illinois USA
November, 2022.

    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2022 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 26/02/2023
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [1]:
import h5py
import numpy as np
import scipy.optimize, scipy.integrate, scipy.interpolate
import matplotlib.pyplot as plt
%matplotlib notebook

## 1. Set up functions for signal trapping using model results from step 2.

We will make some assumptions about good alignment of the canister etc, and can correct these later if they turn out to be incorrect.

In [2]:
def secant(x,r=1.0):
    ''' 
        Secant of a circle radius r at distance x from the centerline.
        Returns zero when outside the radius.
    '''
    s = np.zeros_like(x)
    s[np.abs(x)<=r] = 2*np.sqrt(r**2 - x[np.abs(x)<=r]**2)
    return s

def secantAnnulus(x,ri,ro):
    '''
        Secant of an annulus with inner radius ri and outer ro, at
        distance x from the centerline.
        Returns zero when outside the radius
    '''
    s = secant(x,ro)
    s -= secant(x,ri)
    return s

def secantPartialInsideCircle(x,z,R,rDet,thetaDet):
    '''
        Find the length of a segment of a secant from point (x,z) inside a circle of radius R
        to a detector outside the circle at radius rDet and angle thetaDet from the origin.
        
        This function can accept 1D vectors for x and z.
    '''
    rA = np.sqrt(x**2 + z**2)
    
    sd = np.sqrt ( (rDet*np.cos(thetaDet) - x)**2 + (rDet*np.sin(thetaDet) - z)**2 )
    sdQ= np.sqrt ( (rDet*np.cos(thetaDet) + x)**2 + (rDet*np.sin(thetaDet) + z)**2 )    

    cosGamma = (-rDet**2 + x**2 + z**2 + sdQ**2)/(2*sdQ*rA)
    qb = 2*cosGamma*rA
    qc = x**2 + z**2 - R**2

    
    # solve gfsq avoiding imaginary solutions
    de = (qb**2)/4 - qc
    sdi = (-qb/2 + np.sqrt(np.abs(de))) * (de>=0) 
    #sdi2 = (-qb/2 - np.sqrt(np.abs(de))) * (de>=0) 
    
    #sdi = np.nanmax(np.vstack((sdi1,sdi2)),axis=0)
    
    # no negative solutions allowed
    sd[sd<0]=0
    sdi[sdi<0]=0 
    
    # zero solutions outside the circle
    sdi[rA>R]=0
    
    return sd, sdi

def secantRayProjector(x,z,R,rDet,thetaDet,rayResolutionPts=150):
    '''
        For points outside the circle, project a ray and determine the path length of secant that
        passes through the circle. int rayResolutionPts set the number of points along each ray.
        
        This function can accept 1D vectors for x and z.
    '''
    # find ray from point x,z to detector and check if it passes thru the circle
    zDet = rDet*np.sin(thetaDet)
    xDet = rDet*np.cos(thetaDet)
    m = (rDet*np.sin(thetaDet) - z)/(rDet*np.cos(thetaDet) - x)
    c = z - m*x
    xp = np.linspace(x,xDet,rayResolutionPts)
    zp = m*xp + c
    rp = np.sqrt(xp**2 + zp**2)
    rayData = [xp,zp,rp<=R]
    
    sd,sdi = secantPartialInsideCircle(xp,zp,R,rDet,thetaDet)
    
    # set points outside circle with rays passing thru circle to the max value along the ray.
    rA = np.sqrt(x**2 + z**2)
    outsideCircle = rA>R
    sdi_outside = np.zeros_like(x)
    #print(sdi_outside.shape, sdi.shape)
    sdi_outside[outsideCircle] = np.nanmax(sdi[:,outsideCircle],axis=0)
    
    return sdi_outside, rayData
    
def secantPathLengthWrapper(x,z,R,rDet,thetaDet,rayResolutionPts=150):
    ''' 
        A wrapper function to compute the internal and external part of the projected path length
        and add them together. Other returned values are passed through.
        
        This function can accept 1D vectors for x and z.
    '''
    sd,sdi=secantPartialInsideCircle(x,z,R,rDet,thetaDet)
    sdi_outside,rayData=secantRayProjector(x,z,R,rDet,thetaDet,rayResolutionPts)
    return sdi+sdi_outside, sd, rayData

def calcR2(f, xdata, ydata, popt):
    residuals = ydata- f(xdata, *popt)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    return 1 - (np.nansum(residuals**2) / ss_tot)

In [3]:
def canisterWallExtFn(x, ri, thkExtGradient, thkExtConst, riValve):
    '''
        Calculate absorption through canister wall and rectangular external plastic parts of canister holder
        for transverse co-ordinates x with can aligned at center x0, inner radius ri, wall thickness thk.
        
        Assuming by convention all lengths in mm, and mu is attenuation length in _cm_
    '''
    x0=0
    thkCan=0.706982
    muCan=1.732022
    muExt=0.455318
    
    if (ri<0) | (muCan<0) | (thkExtConst<0) | (muExt<0) | (riValve<0): return 0
    if (ri>12) | (np.abs(thkExtGradient)>1) | (riValve>10): return 0
    
    return secantAnnulus(x-x0,ri,ri+thkCan)*muCan*0.1 +\
           secant(x-x0,riValve)*muCan*0.1 +\
           (thkExtGradient*(x-x0) + thkExtConst - secantAnnulus(x-x0,ri,ri+thkCan))*muExt*0.1 